# Project: Liability-Driven Investment (LDI) Strategy
**Author:** Taha Mohsin

## 1. Executive Summary
- **Business Problem:** The company faces a mandatory schedule of lawsuit payments totaling **$330,000+** over the next 15 years.
- **Objective:** Minimize the *initial* cash required today to fully fund these future liabilities.
- **Methodology:** We modeled a "Cash Flow Matching" strategy using Linear Programming (Gurobi). The model dynamically allocates capital between:
* **High-yield instruments:** Two specific bonds with different maturities and coupon rates.
* **Liquidity instruments:** A savings account (4% yield) to bridge gaps between bond coupon dates and payment deadlines.

---

## 2. The Financial Parameters
The model is constrained by the following payment schedule and investment options.

**Table 1: Mandatory Liability Schedule (The "Cash Out" Requirements)**
| Year | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Payment ($k)** | 10 | 11 | 12 | 14 | 15 | 17 | 19 | 20 | 22 | 24 | 26 | 29 | 31 | 33 | 36 |

**Table 2: Available Investment Instruments (The "Cash In" Sources)**
| Option | Price | Annual Coupon | Maturity | Principal at Maturity | Yield Calculation |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **Bond 1** | $980 | $60 | 5 Years | $1,000 | ~6.5% eff. yield |
| **Bond 2** | $965 | $65 | 12 Years | $1,000 | ~6.9% eff. yield |
| **Savings** | N/A | 4.0% Interest | 1 Year | N/A | Highly Liquid |

In [2]:
# Import the Gurobi library
import gurobipy as gp
from gurobipy import GRB

# --- Problem Data ---
# The required payments (in $1,000s) for each year from 0 to 14
payments = [10, 11, 12, 14, 15, 17, 19, 20, 22, 24, 26, 29, 31, 33, 36]
num_years = len(payments)

# --- Model Creation ---
m = gp.Model("Lawsuit_Funding_Solution_High_Precision")

# --- Decision Variables ---
C0 = m.addVar(name="Initial_Cost")
B1 = m.addVar(name="Bond_1_Quantity")
B2 = m.addVar(name="Bond_2_Quantity")
S = m.addVars(range(num_years - 1), name="Savings")

# --- Objective Function ---
m.setObjective(C0, GRB.MINIMIZE)

# --- Constraints ---
# Year 0
m.addConstr(C0 >= 980 * B1 + 965 * B2 + payments[0] + S[0], "Cash_Balance_Y0")
# Years 1 to 4
for t in range(1, 5):
    m.addConstr(60 * B1 + 65 * B2 + 1.04 * S[t-1] >= payments[t] + S[t], f"Cash_Balance_Y{t}")
# Year 5 (Bond 1 matures)
m.addConstr((60 + 1000) * B1 + 65 * B2 + 1.04 * S[4] >= payments[5] + S[5], "Cash_Balance_Y5")
# Years 6 to 11
for t in range(6, 12):
    m.addConstr(65 * B2 + 1.04 * S[t-1] >= payments[t] + S[t], f"Cash_Balance_Y{t}")
# Year 12 (Bond 2 matures)
m.addConstr((65 + 1000) * B2 + 1.04 * S[11] >= payments[12] + S[12], "Cash_Balance_Y12")
# Year 13
m.addConstr(1.04 * S[12] >= payments[13] + S[13], "Cash_Balance_Y13")
# Year 14
m.addConstr(1.04 * S[13] >= payments[14], "Cash_Balance_Y14")

# --- Optimize the Model ---
m.optimize()

# --- Display the Results ---
if m.Status == GRB.OPTIMAL:
    print("--- Optimal Solution Found ---")
    
    # CHANGED: Increased precision to 8 decimal places for all outputs
    print(f"\n🎯 Minimum initial investment required (Objective): ${m.ObjVal:.8f} (in thousands)")
    print(f"   (This is equal to ${m.ObjVal * 1000:,.2f} in actual dollars)") # Actual dollars kept to 2 decimal places
    
    print("\n✅ Optimal Investment Decisions:")
    print(f"  - Purchase {B1.X:.8f} units of Bond 1.")
    print(f"  - Purchase {B2.X:.8f} units of Bond 2.")
    
    print("\n💰 Savings Account Deposits (in thousands):")
    has_savings = False
    for t in range(num_years - 1):
        if S[t].X > 1e-6: # Using a smaller threshold for high precision check
            print(f"  - At start of Year {t}: Deposit ${S[t].X:.8f}")
            has_savings = True
    if not has_savings:
        print("  - No deposits to the savings account are necessary.")
else:
    print("Optimization was not successful. Status code:", m.Status)

Set parameter Username
Set parameter LicenseID to value 2702254
Academic license - for non-commercial use only - expires 2026-09-02
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: Snapdragon(R) X - X126100 - Qualcomm(R) Oryon(TM) CPU, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 17 columns and 48 nonzeros
Model fingerprint: 0x5a325187
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 12 rows, 14 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   1.796586e+01   0.000000e+00      0s
       5    1.9568367e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optima

## 3. Scenario B: Integer Constraints (Real-World Trading)
"In the previous model, we allowed fractional bond purchases (e.g., buying 0.09 bonds). In this scenario, we restrict the model to 'Integer Constraints' to reflect real-world trading lots where bonds must be bought in whole units."

In [3]:
# Import the Gurobi library
import gurobipy as gp
from gurobipy import GRB

# --- Problem Data ---
# The required payments (in $1,000s) for each year from 0 to 14
payments = [10, 11, 12, 14, 15, 17, 19, 20, 22, 24, 26, 29, 31, 33, 36]
num_years = len(payments)

# --- Model Creation ---
m = gp.Model("Lawsuit_Funding_Integer_Bonds")

# --- Decision Variables ---
# C0 and S[t] are continuous variables (the default)
C0 = m.addVar(name="Initial_Cost")
S = m.addVars(range(num_years - 1), name="Savings")

# B1 and B2 must now be Integers
# We specify this using the vtype=GRB.INTEGER attribute
B1 = m.addVar(name="Bond_1_Quantity", vtype=GRB.INTEGER)
B2 = m.addVar(name="Bond_2_Quantity", vtype=GRB.INTEGER)

# --- Objective Function ---
m.setObjective(C0, GRB.MINIMIZE)

# --- Constraints ---
# The cash flow constraints are identical to the previous model.
# Year 0
m.addConstr(C0 >= 980 * B1 + 965 * B2 + payments[0] + S[0], "Cash_Balance_Y0")
# Years 1 to 4
for t in range(1, 5):
    m.addConstr(60 * B1 + 65 * B2 + 1.04 * S[t-1] >= payments[t] + S[t], f"Cash_Balance_Y{t}")
# Year 5 (Bond 1 matures)
m.addConstr((60 + 1000) * B1 + 65 * B2 + 1.04 * S[4] >= payments[5] + S[5], "Cash_Balance_Y5")
# Years 6 to 11
for t in range(6, 12):
    m.addConstr(65 * B2 + 1.04 * S[t-1] >= payments[t] + S[t], f"Cash_Balance_Y{t}")
# Year 12 (Bond 2 matures)
m.addConstr((65 + 1000) * B2 + 1.04 * S[11] >= payments[12] + S[12], "Cash_Balance_Y12")
# Year 13
m.addConstr(1.04 * S[12] >= payments[13] + S[13], "Cash_Balance_Y13")
# Year 14
m.addConstr(1.04 * S[13] >= payments[14], "Cash_Balance_Y14")

# --- Optimize the Model ---
m.optimize()

# --- Display the Results ---
if m.Status == GRB.OPTIMAL:
    print("--- Optimal Solution with Integer Bonds Found ---")
    
    print(f"\n🎯 Minimum initial investment required (Objective): ${m.ObjVal:,.2f} (in thousands)")
    print(f"   (This is equal to ${m.ObjVal * 1000:,.2f} in actual dollars)")
    
    print("\n✅ Optimal Investment Decisions:")
    # The .X attribute will now be a whole number for B1 and B2
    print(f"  - Purchase {B1.X:.0f} units of Bond 1.")
    print(f"  - Purchase {B2.X:.0f} units of Bond 2.")
    
    print("\n💰 Savings Account Deposits (in thousands):")
    has_savings = False
    for t in range(num_years - 1):
        if S[t].X > 1e-6:
            print(f"  - At start of Year {t}: Deposit ${S[t].X:,.4f}")
            has_savings = True
    if not has_savings:
        print("  - No deposits to the savings account are necessary.")
else:
    print("Optimization was not successful. Status code:", m.Status)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: Snapdragon(R) X - X126100 - Qualcomm(R) Oryon(TM) CPU, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 17 columns and 48 nonzeros
Model fingerprint: 0xa5b9badc
Variable types: 15 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Presolve removed 15 rows and 17 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 230.437 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.304370405176e+02, best bound 2.304370405176e+02, gap 0.0000%
--- Optimal Solution with Integer Bonds Found ---

🎯 Minimum initial

## 4. Managerial Insights & Conclusion
The optimization reduced the capital requirement to **$195,683** (Base Scenario) vs **$230,437** (Integer Scenario).

**Key Strategy Drivers:**
1.  **Bond Ladders:** The model heavily utilizes Bond 1 and Bond 2 to cover the "expensive" years (Years 5-12).
2.  **Liquidity Management:** It does not simply hold cash. It actively deposits surplus coupons into the Savings account in early years (e.g., ~$4.8k in Year 0) to accrue 4% interest, withdrawing it precisely when the lawsuit payments spike in Years 13-14.